In [1]:
import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint
from datetime import datetime
import random
from feature_engine.imputation import MeanMedianImputer
from sklearn.cluster import KMeans

In [2]:
project_name = 'Okhttp'
commit_project_name = 'Okhttp'

In [3]:
okhttp_commits_df = pd.read_csv(f'raw_commits_github_json/{project_name}/{commit_project_name}_commits.csv')

In [4]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring
0,3ef564b0ddcb00655e540e52ad882a5881c1dc54,Jesse Wilson,jesse@swank.ca,2021-11-01 13:29:31,swankjesse,Don't document mockwebserver3.internal (#6895),1ed9863131c87420381d8a6be48b96157a9bb764,2021-10-29 07:39:56,77.0,0
1,1ed9863131c87420381d8a6be48b96157a9bb764,Jesse Wilson,jesse@swank.ca,2021-10-29 07:39:56,swankjesse,Upgrade to Okio 3.0.0 (#6896),bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,2021-10-22 16:51:49,158.0,0
2,bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,Jim Hurne,hurne_jim@yahoo.com,2021-10-22 16:51:49,jthurne,Fix EventSources.processResponse to not throw ...,1ff16232f86d2b302840a3bb2f9d386e94374753,2021-10-15 05:04:08,179.0,0
3,1ff16232f86d2b302840a3bb2f9d386e94374753,Clément Dessoude,clement.dessoude@gmail.com,2021-10-15 05:04:08,clementdessoude,fix(docs): update broken link on docs homepage...,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,2021-10-01 01:13:35,339.0,0
4,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,Jesse Wilson,jwilson@squareup.com,2021-10-01 01:13:35,swankjesse,Changelog for OkHttp 4.9.2,286187102848b30d9580794795746b28462a79d0,2021-09-24 06:13:58,162.0,0


In [5]:
len(okhttp_commits_df)

4593

In [6]:
okhttp_commits_df = okhttp_commits_df[okhttp_commits_df['time_taken'] > 0]
okhttp_commits_df = okhttp_commits_df[okhttp_commits_df['time_taken'] < 168]
len(okhttp_commits_df)

3062

In [7]:
okhttp_commits_df['message'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [8]:
h2o.init(nthreads=4)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 12.0.1+12, mixed mode, sharing)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\tanji\AppData\Local\Temp\tmptosjru5w
  JVM stdout: C:\Users\tanji\AppData\Local\Temp\tmptosjru5w\h2o_tanji_started_from_python.out
  JVM stderr: C:\Users\tanji\AppData\Local\Temp\tmptosjru5w\h2o_tanji_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Kuala_Lumpur
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.7
H2O_cluster_version_age:,3 months and 15 days !!!
H2O_cluster_name:,H2O_from_python_tanji_eafnu9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.984 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [751]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [752]:
STOP_WORDS = stopwords.words('english')
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]
STOP_WORDS = ['']



In [753]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [754]:
def create_token_table(data_hdf, col_name, prefix, sent_sample_rate = 0.0, epochs = 25, vec_size=20):
    # Tokenize
    words = tokenize(data_hdf[col_name])
    w2v_model = H2OWord2vecEstimator(sent_sample_rate = sent_sample_rate, 
                                     epochs = epochs, 
                                     vec_size = vec_size)
    w2v_model.train(training_frame=words)
    
    # Create the training vectors
    vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
    
    
    # Prefix naming the columns
    vecs.columns = ['%s%s' % (prefix, c) for c in vecs.columns]
    data2_hdf = data_hdf.cbind(vecs)
    return data2_hdf, w2v_model

    
#     data3_hdf = data2_hdf[mask,:].cbind(vecs[mask,:])

In [755]:
commit_message_df = okhttp_commits_df[['message',]]

In [756]:
commit_message_df.shape

(3062, 1)

In [757]:
commit_message_df.head()

,message
0,Don't document mockwebserver3.internal (#6895)
1,Upgrade to Okio 3.0.0 (#6896)
4,Changelog for OkHttp 4.9.2
5,JDK 17 CI Build (#6864)
6,Update dependencies including kotlin 1.5.31 (#...


In [758]:
commit_message_hdf = h2o.H2OFrame(commit_message_df, 
                         column_types = {'message': 'string'}
                        )
commit_message_hdf.types

Parse progress: |█████████████████████████████████████████████████████████| 100%


{'message': 'string'}

In [759]:
commit_message_hdf.head()

message
Don't document mockwebserver3.internal (#6895)
Upgrade to Okio 3.0.0 (#6896)
Changelog for OkHttp 4.9.2
JDK 17 CI Build (#6864)
Update dependencies including kotlin 1.5.31 (#6865)
Update dependencies (#6850)
Replace deprecated method used in README (#6863)
Specify languages in kdoc code fragments (#6844)
"Fixed leading spaces for code blocks in recipes.md Markdown coverts text with four leading spaces into a code block. Fenced code blocks in this file had four leading spaces in addition to the being wrapped with ```, because of this the code blocks weren't being rendered properly in the markdown preview. Removing the two out of the four leading spaces fixed the issue."
Declare types & Cleanup (#6842) * Declare types * Code cleanup * Declare types


In [760]:
# Train owner names model
commit_message_vecs_hdf, commit_message_w2v_model = create_token_table(commit_message_hdf,
                                                                  'message',
                                                                   prefix='message_',
                                                                   vec_size=30)

'''
words = tokenize(commit_message_hdf['message'])
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 20, 
                                 vec_size = 30)
w2v_model.train(training_frame=words)

# Create the training vectors
vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")


# Prefix naming the columns
vecs.columns = ['%s%s' % ('message_', c) for c in vecs.columns]
data2_hdf = commit_message_hdf.cbind(vecs)
print(data2_hdf)

wv_path = 'models/'
model_path = h2o.save_model(model = w2v_model, path= wv_path ,force=True)

#Load Model
w2v_model2 = h2o.load_model(model_path)
'''

word2vec Model Build progress: |██████████████████████████████████████████| 100%


'\nwords = tokenize(commit_message_hdf[\'message\'])\nw2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 20, \n                                 vec_size = 30)\nw2v_model.train(training_frame=words)\n\n# Create the training vectors\nvecs = w2v_model.transform(words, aggregate_method = "AVERAGE")\n\n\n# Prefix naming the columns\nvecs.columns = [\'%s%s\' % (\'message_\', c) for c in vecs.columns]\ndata2_hdf = commit_message_hdf.cbind(vecs)\nprint(data2_hdf)\n\nwv_path = \'models/\'\nmodel_path = h2o.save_model(model = w2v_model, path= wv_path ,force=True)\n\n#Load Model\nw2v_model2 = h2o.load_model(model_path)\n'

In [761]:
commit_message_w2v_model.find_synonyms('refactor')

OrderedDict([('connectionspec', 0.575116753578186),
             ('okhttpclient', 0.5565364360809326),
             ('slightly', 0.5478174686431885),
             ('copyright', 0.5450999736785889),
             ('wrap', 0.5271931886672974),
             ('flags', 0.5219516158103943),
             ('converting', 0.5149720907211304),
             ('urls', 0.5104042291641235),
             ('formencodingbuilder', 0.49869126081466675),
             ('realconnection', 0.4962990880012512),
             ('codes', 0.4962385296821594),
             ('push', 0.4954840838909149),
             ('framed', 0.48726487159729004),
             ('mock', 0.48528122901916504),
             ('huffman', 0.47972819209098816),
             ('migrate', 0.47235020995140076),
             ('tweaks', 0.4679509699344635),
             ('kotlin', 0.4675340950489044),
             ('import', 0.4633508622646332),
             ('introduces', 0.4548821449279785)])

In [762]:
rand_list = sorted( random.sample(range(commit_message_vecs_hdf.nrow), 4 ) )
commit_message_vecs_hdf[rand_list, :]

message,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,message_C10,message_C11,message_C12,message_C13,message_C14,message_C15,message_C16,message_C17,message_C18,message_C19,message_C20,message_C21,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30
Fix MockWebServer handling of 'Expect: 100 Continue' (#3563) Previously we'd skip the request body unless the socket policy was EXPECT_CONTINUE. Closes: https://github.com/square/okhttp/issues/3498,-0.337229,0.0659416,0.177421,-0.0656772,-0.057497,0.0365295,-0.239664,0.0585626,-0.266663,0.109542,0.0434399,-0.139051,-0.0708526,0.141562,0.712798,0.176578,0.0745481,-0.23233,-0.0212519,-0.323362,0.153994,0.00166268,0.13489,-0.221936,-0.140416,-0.00908048,0.144335,-0.220499,-0.0462521,0.267398
Merge pull request #911 from square/jw/much-sadness Revert builder-esque return types from OkHttpClient.,0.0849861,-0.0748806,0.163995,0.059946,0.0436476,0.0854602,0.197185,-0.328798,-0.353378,0.358439,-0.241369,-0.170544,0.138871,-0.00815943,0.759145,0.244609,-0.229463,0.280921,-0.212851,0.0563127,0.105981,-0.0179336,-0.14501,-0.0558872,0.0385127,-0.173665,-0.287131,-0.389245,-0.480659,0.156383
"Track Connection ownership. Connection ownership is somewhat racy: when an HttpURLConnection reads the last byte of a connection, its ownership is released and given to the connection pool.",0.00781533,0.0709192,0.217108,-0.202506,0.181123,0.313964,-0.167146,0.265196,0.0556147,-0.0200286,0.164043,0.0412175,-0.0626822,0.121656,0.342108,0.0272494,-0.311618,-0.207882,0.109006,-0.164078,0.102669,0.0271955,0.107125,-0.0012475,-0.088859,-0.166846,0.514191,-0.20056,-0.256745,0.252965
Add syntax highlighting to changes,-0.0663084,-0.416265,0.325223,0.366134,0.377555,0.112998,0.0556358,0.363823,-0.600332,0.508335,0.133513,-0.039212,0.00121917,0.19351,0.535738,-0.0435368,0.00782221,0.532076,0.0130202,0.43204,0.409913,-0.128306,0.0154781,0.10689,0.100071,0.523786,-0.469418,-0.52595,-0.115812,-0.0213746


In [763]:
kmean = KMeans(n_clusters=15)

In [764]:
commit_message_vecs_hdf = commit_message_vecs_hdf.as_data_frame()

In [765]:
commit_message_vecs_hdf = commit_message_vecs_hdf.fillna(0)

In [766]:
commit_message_vecs_hdf.iloc[:,1:]

,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,message_C10,...,message_C21,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30
0,0.066921,0.189805,0.496224,-0.295545,0.121561,0.096789,0.461610,0.012689,-0.165712,-0.161842,...,0.333319,0.284282,0.695765,-0.282829,0.291704,0.071098,0.186469,-0.465448,-0.144302,-0.053959
1,-0.224503,0.493227,0.327968,-0.046558,0.013304,0.626669,-0.360213,-0.407694,-0.761733,0.136012,...,0.411316,0.098395,0.307152,0.318662,0.443493,0.278862,0.046532,-0.126054,-0.315776,-0.235760
2,-0.254621,0.489618,0.432759,0.098308,0.050692,0.251910,-0.057483,0.234733,-0.524723,-0.096793,...,-0.018100,-0.016466,0.226919,-0.126392,-0.009719,-0.034013,-0.099595,-0.534876,0.306692,0.114519
3,0.096390,0.364624,-0.512927,-0.476710,-0.770989,0.511780,-0.360884,-0.091614,-1.014498,-0.469576,...,0.974566,-0.118128,-0.389677,-0.161243,0.030677,0.534723,-0.339640,-0.108255,-1.153880,-0.389488
4,-0.322107,0.346197,0.213128,-0.221670,0.337336,0.004349,-0.447457,-0.023277,-0.924667,-0.009446,...,0.071726,0.252725,0.288440,0.166178,0.617369,0.187167,-0.119834,-0.519278,-0.435319,-0.182095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3057,-0.054474,0.153250,-0.307423,-0.333777,-0.476985,0.193937,-0.098531,-0.058224,-0.611226,-0.057294,...,0.482531,-0.305667,-0.359519,-0.130729,0.056389,0.237035,-0.116856,-0.245350,-0.654557,-0.314939
3058,0.004905,-0.145791,-0.003233,-0.441805,-0.231505,0.254123,0.185959,0.064678,-0.406620,0.095307,...,0.712594,0.133811,-0.263652,0.637302,-0.372918,0.321805,-0.730672,-0.563456,-0.186300,-0.748636
3059,-0.164942,-0.271502,0.055014,-0.219573,-0.299604,0.277143,-0.004388,0.020125,-0.383072,0.170975,...,0.364967,-0.753522,-0.629206,-0.111487,-0.074579,0.140584,0.096749,-0.332886,-0.324713,-0.405688
3060,0.608618,0.260310,-0.007356,0.261989,0.300822,0.191185,-0.218408,0.103289,0.081853,0.006801,...,0.216397,0.046335,0.130569,-0.139444,-0.108551,-0.047958,0.330484,-0.361351,-0.195691,-0.084337


In [767]:
kmean.fit(commit_message_vecs_hdf.iloc[:,1:])

KMeans(n_clusters=15)

In [768]:
kmean.labels_

array([ 7,  9, 11, ...,  6,  7,  0])

In [769]:
print(len(kmean.labels_))

3062


In [770]:
commit_message_vecs_hdf['kmean_label'] = kmean.labels_

In [771]:
commit_message_vecs_hdf.head()

,message,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,...,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30,kmean_label
0,Don't document mockwebserver3.internal (#6895),0.066921,0.189805,0.496224,-0.295545,0.121561,0.096789,0.461610,0.012689,-0.165712,...,0.284282,0.695765,-0.282829,0.291704,0.071098,0.186469,-0.465448,-0.144302,-0.053959,7
1,Upgrade to Okio 3.0.0 (#6896),-0.224503,0.493227,0.327968,-0.046558,0.013304,0.626669,-0.360213,-0.407694,-0.761733,...,0.098395,0.307152,0.318662,0.443493,0.278862,0.046532,-0.126054,-0.315776,-0.235760,9
2,Changelog for OkHttp 4.9.2,-0.254621,0.489618,0.432759,0.098308,0.050692,0.251910,-0.057483,0.234733,-0.524723,...,-0.016466,0.226919,-0.126392,-0.009719,-0.034013,-0.099595,-0.534876,0.306692,0.114519,11
3,JDK 17 CI Build (#6864),0.096390,0.364624,-0.512927,-0.476710,-0.770989,0.511780,-0.360884,-0.091614,-1.014498,...,-0.118128,-0.389677,-0.161243,0.030677,0.534723,-0.339640,-0.108255,-1.153880,-0.389488,1
4,Update dependencies including kotlin 1.5.31 (#...,-0.322107,0.346197,0.213128,-0.221670,0.337336,0.004349,-0.447457,-0.023277,-0.924667,...,0.252725,0.288440,0.166178,0.617369,0.187167,-0.119834,-0.519278,-0.435319,-0.182095,9


In [772]:
group_list = sorted(commit_message_vecs_hdf.kmean_label.unique())
for group_id in group_list:
    mask = commit_message_vecs_hdf.kmean_label == group_id
    print('group_id = %d' % group_id)
    min_size = min(sum(mask), 10)
    print( commit_message_vecs_hdf[mask].sample(min_size) )

group_id = 0
                                                message  message_C1  \
962   Merge pull request #3001 from square/jwilson.1...    0.145949   
2235  Merge pull request #619 from square/jwilson_03...    0.057390   
1872  Merge pull request #1234 from square/jw/musing...    0.142490   
354   Merge pull request #4654 from square/jwilson.0...    0.042785   
390   Merge pull request #4609 from square/jwilson.0...    0.122522   
2986  Merge pull request #5159 from square/jwilson.0...   -0.144959   
1535  Merge pull request #1708 from square/jw/framed...   -0.006923   
2739  Merge pull request #5718 from square/jwilson.0...    0.020711   
1981  Merge pull request #1079 from square/jwilson_1...    0.145501   
1649  Merge pull request #1618 from square/jwilson_0...    0.036088   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
962    -0.439789    0.298918   -0.356508   -0.207362    0.164085    0.154901   
2235   -0.148555    0.167403   -0.114710    0

[10 rows x 32 columns]
group_id = 3
                                                message  message_C1  \
2782  Merge pull request #5618 from sullis/gradle-6....    0.184911   
1662  Merge pull request #1598 from artem-zinnatulli...   -0.062739   
266   Merge pull request #4769 from amirlivneh/fix-e...   -0.008022   
2891  Merge pull request #5368 from yschimke/cleanup...    0.143224   
2327  Merge pull request #593 from nfuller/FixImport...    0.006728   
616   Merge pull request #4046 from yschimke/conscry...    0.087401   
336   Merge pull request #4672 from amirlivneh/verif...    0.247254   
2881  Merge pull request #5396 from thescouser89/doc...   -0.088387   
617   Merge pull request #4072 from yschimke/better_...    0.195355   
2552       Merge remote-tracking branch 'origin/master'    0.024508   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
2782   -0.254654    0.285373   -0.114586   -0.646267   -0.002721   -0.240337   
1662   -0.406406    0.

[10 rows x 32 columns]
group_id = 6
                                                message  message_C1  \
2248                  Document 4.x requirements (#5897)    0.416667   
2601                              Fix checkstyle errors   -0.239727   
2414                                      timeout issue    0.079152   
470   Enable cancelWithStreamNotCompleted test\r\n\r...    0.135515   
2869               Hows My Ssl test for Android (#5428)   -0.141596   
1630                               Small Javadoc fixes.    0.194476   
629                        Correct the javadoc code tag    0.440582   
1897            Update MWS test style to match project.   -0.348771   
640                                       Conscrypt 1.1    0.121225   
717   Failed, second connection and call.enqueue Eve...    0.144369   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
2248    0.110242    0.542154   -0.290342   -0.095461    0.107110    0.985982   
2601   -0.255717    0.

[10 rows x 32 columns]
group_id = 9
                                                message  message_C1  \
196                          Upgrade CircleCI to 11.0.3   -0.373809   
2028  Merge pull request #952 from andrewgaul/bouncy...   -0.044738   
240   Okio upgrade post kotlin (#4814)\r\n\r\nOkio 2...   -0.171301   
1525                                Bump Okio to 1.5.0.    0.092283   
175                          Gradle 5.4 upgrade (#4959)   -0.393591   
566                        Upgrade to BouncyCastle 1.60   -0.476124   
62                      Conscrypt 2.5.2 Upgrade (#6613)   -0.329682   
36                   Android dependency upgrade (#6755)   -0.438867   
2728                                         Gradle 6.1   -0.006593   
528              Upgrade Conscrypt to 1.4.0 (was 1.3.0)   -0.228580   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
196     0.175793    0.372568   -0.226796   -0.161767    0.790919   -0.229924   
2028    0.147553    0.

[10 rows x 32 columns]
group_id = 12
                                                message  message_C1  \
1925  Merge pull request #1162 from square/jw/http-2...   -0.149641   
1299  Merge pull request #2184 from square/jwilson_0...    0.290544   
2625  Merge pull request #77 from square/jwilson/hea...    0.000261   
774   Merge pull request #3350 from ericaschulz/immu...   -0.041369   
1144  Merge branch 'adennie-log-connect-failure'\r\n...    0.199002   
891   Merge pull request #3208 from chhsiao90/http2-...   -0.238115   
2643  Merge pull request #67 from square/jwilson/lib...    0.035717   
2670  Merge pull request #46 from square/jwilson/clo...   -0.053044   
1824  Merge pull request #1279 from square/jw/public...   -0.197280   
1181  Merge pull request #2446 from square/jwilson_0...    0.048504   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
1925   -0.382708    0.192395   -0.096114    0.008526    0.027024   -0.036486   
1299   -0.323678   -0

[10 rows x 32 columns]


In [773]:
print(len(okhttp_commits_df))
print(len(commit_message_vecs_hdf))

3062
3062


In [774]:
okhttp_commits_df['kmean_label'] = kmean.labels_
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,kmean_label
0,3ef564b0ddcb00655e540e52ad882a5881c1dc54,Jesse Wilson,jesse@swank.ca,2021-11-01 13:29:31,swankjesse,Don't document mockwebserver3.internal (#6895),1ed9863131c87420381d8a6be48b96157a9bb764,2021-10-29 07:39:56,77.0,0,7
1,1ed9863131c87420381d8a6be48b96157a9bb764,Jesse Wilson,jesse@swank.ca,2021-10-29 07:39:56,swankjesse,Upgrade to Okio 3.0.0 (#6896),bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,2021-10-22 16:51:49,158.0,0,9
4,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,Jesse Wilson,jwilson@squareup.com,2021-10-01 01:13:35,swankjesse,Changelog for OkHttp 4.9.2,286187102848b30d9580794795746b28462a79d0,2021-09-24 06:13:58,162.0,0,11
5,286187102848b30d9580794795746b28462a79d0,Yuri Schimke,yuri@schimke.ee,2021-09-24 06:13:58,yschimke,JDK 17 CI Build (#6864),980dfbbe9d0e276b0a103df373f8969e21667e50,2021-09-20 19:37:03,82.0,0,1
6,980dfbbe9d0e276b0a103df373f8969e21667e50,Goooler,wangzongler@gmail.com,2021-09-20 19:37:03,Goooler,Update dependencies including kotlin 1.5.31 (#...,6f328ecab46e070345bf6f4de86b98a24e50e3d8,2021-09-16 06:33:06,109.0,0,9


In [775]:
okhttp_commits_df['kmean_label'].value_counts()

12    566
2     540
0     413
7     311
3     248
6     202
10    184
14    158
13    119
5      80
11     74
1      61
9      46
8      36
4      24
Name: kmean_label, dtype: int64

In [776]:
okhttp_commits_df['kmean_label'].isna().sum()

0

In [777]:
okhttp_commits_df[okhttp_commits_df['contains_refactoring'] == 1]

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,kmean_label
119,b84627ef4bc43f2096c1b764720f9ff2387dca32,Jesse Wilson,jwilson@squareup.com,2021-01-30 19:44:19,swankjesse,Prepare for release 5.0.0-alpha.2.,db229c8cf500261a4fd76b292759585619a82bb8,2021-01-30 17:57:23,1.0,1,4
143,083315a473d8b2fc972a70449272b296683ac38c,Yuri Schimke,yuri@schimke.ee,2020-11-21 13:28:45,yschimke,Launcher for quick tests (#6437),ec1d18d143a5d4630629c68f7bfbe2691639d90a,2020-11-21 09:50:01,3.0,1,6
150,ff959f8fd550b8f55a47038ea4235b1b2ed2eaa8,Yuri Schimke,yuri@schimke.ee,2020-11-15 06:42:22,yschimke,Android Additional Tests (#6420),4adcb33d1b859e589e929629232e2e85092948d2,2020-11-14 17:38:22,13.0,1,6
152,4036fa55b72faaad3a14f806094184ade157917d,Yuri Schimke,yuri@schimke.ee,2020-11-14 17:21:04,yschimke,Update remaining junit4 tests (#6421),5bd0a88ff3701737c10573a8a36ca4b1e00e6d09,2020-11-14 11:59:53,5.0,1,11
191,59ae0c138d6468aac5d92781b19cddc26e04e862,Jesse Wilson,jesse@swank.ca,2020-11-01 05:46:35,swankjesse,Convert more tests to JUnit 5 (#6366),963c76864344d0d55ba7421f474c130a958a9649,2020-10-31 22:51:34,6.0,1,7
192,963c76864344d0d55ba7421f474c130a958a9649,Jesse Wilson,jesse@swank.ca,2020-10-31 22:51:34,swankjesse,Migrate all OkHttpClientTestRule uses to JUnit...,559b0f8d4b771e30ea27cd5712f55ae092f85aca,2020-10-31 13:39:47,9.0,1,7
1186,f8065acac28a0b86f3bdbd854ac87af0789416d0,Jesse Wilson,jesse@swank.ca,2020-10-30 06:59:02,swankjesse,Execute CallTest with JUnit 5 (#6349)\n\n* Exe...,f1bc9d1d3339d0696e232a5d8a2d7830630a8242,2020-10-24 04:03:17,146.0,1,7
1190,3e331c108905a97fa9718b40844ddc1356fc86b5,Jesse Wilson,jesse@swank.ca,2020-10-18 12:13:09,swankjesse,Rename okhttp3.mockwebserver to mockwebserver3...,02ac2f38e8f8dacebb090dcf7f7eb30ed29d4fc7,2020-10-18 11:02:17,1.0,1,14
1212,334eb0072512155e57a809b2fa18861c0d68ec9f,Jesse Wilson,jesse@swank.ca,2020-10-02 01:46:58,swankjesse,Rename okhttp3.internal.sse to okhttp3.sse.int...,480c20e46bb1745e280e42607bbcc73b2c953d97,2020-10-02 00:07:44,1.0,1,7


In [778]:
grouped_main_commits = okhttp_commits_df.groupby(['kmean_label', 'login']).agg({'time_taken': ['mean', 'min', 'max']})['time_taken'].reset_index()
grouped_main_commits

,kmean_label,login,mean,min,max
0,0,JakeWharton,22.111111,1.0,146.0
1,0,adriancole,41.750000,4.0,117.0
2,0,edenman,33.750000,1.0,93.0
3,0,mescortes,10.000000,2.0,16.0
4,0,oldergod,13.500000,2.0,20.0
...,...,...,...,...,...
292,14,nfuller,124.000000,34.0,164.0
293,14,quinnjn,146.000000,146.0,146.0
294,14,rtyley,2.000000,2.0,2.0
295,14,swankjesse,27.201754,1.0,145.0


In [779]:
grouped_main_commits[grouped_main_commits['kmean_label']==0]

,kmean_label,login,mean,min,max
0,0,JakeWharton,22.111111,1.0,146.0
1,0,adriancole,41.750000,4.0,117.0
2,0,edenman,33.750000,1.0,93.0
3,0,mescortes,10.000000,2.0,16.0
4,0,oldergod,13.500000,2.0,20.0
5,0,pforhan,95.000000,95.0,95.0
6,0,rjrjr,5.000000,5.0,5.0
7,0,swankjesse,22.045139,1.0,138.0


### Percentile functions
To be passed into .agg()

In [780]:
# 10th Percentile
def q10(x):
    return x.quantile(0.1)

# 50th Percentile
def q20(x):
    return x.quantile(0.2)

# 90th Percentile
def q25(x):
    return x.quantile(0.25)

# 90th Percentile
def q30(x):
    return x.quantile(0.30)

# 90th Percentile
def q40(x):
    return x.quantile(0.40)

# 90th Percentile
def q50(x):
    return x.quantile(0.50)

# 90th Percentile
def q60(x):
    return x.quantile(0.60)

# 90th Percentile
def q70(x):
    return x.quantile(0.70)

# 90th Percentile
def q75(x):
    return x.quantile(0.75)

# 90th Percentile
def q80(x):
    return x.quantile(0.8)

# 90th Percentile
def q90(x):
    return x.quantile(0.9)




In [781]:
#okhttp_commits_df_refactored = okhttp_commits_df[okhttp_commits_df['contains_refactoring'] == 1]
okhttp_commits_df_refactored = okhttp_commits_df.copy()
okhttp_commits_df_refactored = okhttp_commits_df_refactored.groupby(['kmean_label']).agg({'time_taken': ['mean', 'min', 'max', 
                                                                                  q10, q20, q25, 
                                                                                  q30, q40, q50, 
                                                                                  q60, q70, q75, 
                                                                                  q80, q90]})

okhttp_commits_df_refactored.columns = ['time_taken_mean',
                                       'time_taken_min',
                                       'time_taken_max',
                                       'time_taken_q10',
                                       'time_taken_q20',
                                       'time_taken_q25',
                                       'time_taken_q30',
                                       'time_taken_q40',
                                       'time_taken_q50',
                                       'time_taken_q60',
                                       'time_taken_q70',
                                       'time_taken_q75',
                                       'time_taken_q80',
                                       'time_taken_q90']
                                       
okhttp_commits_df_refactored

,time_taken_mean,time_taken_min,time_taken_max,time_taken_q10,time_taken_q20,time_taken_q25,time_taken_q30,time_taken_q40,time_taken_q50,time_taken_q60,time_taken_q70,time_taken_q75,time_taken_q80,time_taken_q90
kmean_label,,,,,,,,,,,,,,
0,22.331719,1.0,146.0,1.0,2.0,3.00,4.0,8.0,12.0,17.0,22.0,26.00,32.0,62.0
1,37.245902,1.0,150.0,1.0,3.0,4.00,6.0,9.0,19.0,28.0,57.0,62.00,79.0,92.0
2,28.957407,1.0,166.0,2.0,4.0,5.00,7.0,10.0,15.0,20.0,28.0,35.00,44.0,86.1
3,33.959677,1.0,166.0,2.0,4.0,6.00,8.0,13.0,19.0,26.0,41.8,51.25,62.2,90.3
4,8.666667,1.0,48.0,1.0,1.0,1.00,1.9,2.2,3.5,7.4,10.4,14.00,15.2,19.0
5,13.800000,1.0,59.0,2.0,4.0,4.75,5.0,7.0,11.0,12.0,14.0,15.00,18.2,34.2
6,24.722772,1.0,148.0,2.0,3.0,4.00,6.0,10.4,14.0,20.0,25.0,30.75,35.8,65.8
7,24.990354,1.0,167.0,1.0,2.0,3.00,5.0,8.0,12.0,17.0,22.0,27.50,38.0,72.0
8,19.166667,1.0,136.0,1.0,2.0,3.00,3.5,4.0,5.5,7.0,13.0,24.75,29.0,40.5


In [782]:
final_df = pd.merge(okhttp_commits_df_refactored, okhttp_commits_df, how='left', on=['kmean_label'])
final_df = final_df[final_df['contains_refactoring'] == 1]

In [783]:
final_df['project_name'] = project_name

In [784]:
final_df.head()

,kmean_label,time_taken_mean,time_taken_min,time_taken_max,time_taken_q10,time_taken_q20,time_taken_q25,time_taken_q30,time_taken_q40,time_taken_q50,...,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,project_name
1262,4,8.666667,1.0,48.0,1.0,1.0,1.0,1.9,2.2,3.5,...,Jesse Wilson,jwilson@squareup.com,2021-01-30 19:44:19,swankjesse,Prepare for release 5.0.0-alpha.2.,db229c8cf500261a4fd76b292759585619a82bb8,2021-01-30 17:57:23,1.0,1,Okhttp
1381,6,24.722772,1.0,148.0,2.0,3.0,4.0,6.0,10.4,14.0,...,Yuri Schimke,yuri@schimke.ee,2020-11-21 13:28:45,yschimke,Launcher for quick tests (#6437),ec1d18d143a5d4630629c68f7bfbe2691639d90a,2020-11-21 09:50:01,3.0,1,Okhttp
1385,6,24.722772,1.0,148.0,2.0,3.0,4.0,6.0,10.4,14.0,...,Yuri Schimke,yuri@schimke.ee,2020-11-15 06:42:22,yschimke,Android Additional Tests (#6420),4adcb33d1b859e589e929629232e2e85092948d2,2020-11-14 17:38:22,13.0,1,Okhttp
1586,7,24.990354,1.0,167.0,1.0,2.0,3.0,5.0,8.0,12.0,...,Jesse Wilson,jesse@swank.ca,2020-11-01 05:46:35,swankjesse,Convert more tests to JUnit 5 (#6366),963c76864344d0d55ba7421f474c130a958a9649,2020-10-31 22:51:34,6.0,1,Okhttp
1587,7,24.990354,1.0,167.0,1.0,2.0,3.0,5.0,8.0,12.0,...,Jesse Wilson,jesse@swank.ca,2020-10-31 22:51:34,swankjesse,Migrate all OkHttpClientTestRule uses to JUnit...,559b0f8d4b771e30ea27cd5712f55ae092f85aca,2020-10-31 13:39:47,9.0,1,Okhttp


In [785]:
project_refactoring_data = pd.read_csv(f'Effort_Estimation_Results_CKMetrics/{project_name}.csv')

In [786]:
project_refactoring_data.head()

,file,class,type,cbo,wmc,dit,rfc,lcom,totalMethods,staticMethods,...,modifiers,latest_commit,commit_compared_with,num_dependency,num_line_affected,actual_num_of_classes_touched,latest_commit_date,commit_compared_with_date,author,author_email
0,E:\SoftwareRemodularization\raw_sourcecode\Okh...,okhttp3.sse.internal.Event,class,2,7,1,4,0,4,0,...,16,4fd1e8f99833eebdd2e99f3456322aa197f5e652,334eb0072512155e57a809b2fa18861c0d68ec9f,143.0,31,1,2020-10-07 03:28:26,2020-10-02 01:46:58,Jesse Wilson,jesse@swank.ca
1,E:\SoftwareRemodularization\raw_sourcecode\Okh...,okhttp3.sse.internal.EventSourceHttpTest,class,17,11,1,28,0,10,0,...,17,4fd1e8f99833eebdd2e99f3456322aa197f5e652,334eb0072512155e57a809b2fa18861c0d68ec9f,6.0,123,2,2020-10-07 03:28:26,2020-10-02 01:46:58,Jesse Wilson,jesse@swank.ca
2,E:\SoftwareRemodularization\raw_sourcecode\Okh...,okhttp3.sse.internal.EventSourceRecorder,class,9,16,2,10,15,10,0,...,17,4fd1e8f99833eebdd2e99f3456322aa197f5e652,334eb0072512155e57a809b2fa18861c0d68ec9f,5.0,118,3,2020-10-07 03:28:26,2020-10-02 01:46:58,Jesse Wilson,jesse@swank.ca
3,E:\SoftwareRemodularization\raw_sourcecode\Okh...,okhttp3.sse.internal.ServerSentEventIteratorTest,class,6,19,1,10,0,18,0,...,17,4fd1e8f99833eebdd2e99f3456322aa197f5e652,334eb0072512155e57a809b2fa18861c0d68ec9f,58.0,198,4,2020-10-07 03:28:26,2020-10-02 01:46:58,Jesse Wilson,jesse@swank.ca
4,E:\SoftwareRemodularization\raw_sourcecode\Okh...,okhttp3.sse.internal.Event,class,2,7,1,4,0,4,0,...,16,4fd1e8f99833eebdd2e99f3456322aa197f5e652,334eb0072512155e57a809b2fa18861c0d68ec9f,143.0,31,5,2020-10-07 03:28:26,2020-10-02 01:46:58,Jesse Wilson,jesse@swank.ca


In [787]:
project_refactoring_data.columns

Index(['file', 'class', 'type', 'cbo', 'wmc', 'dit', 'rfc', 'lcom',
       'totalMethods', 'staticMethods', 'publicMethods', 'privateMethods',
       'protectedMethods', 'defaultMethods', 'abstractMethods', 'finalMethods',
       'synchronizedMethods', 'totalFields', 'staticFields', 'publicFields',
       'privateFields', 'protectedFields', 'defaultFields', 'finalFields',
       'synchronizedFields', 'nosi', 'loc', 'returnQty', 'loopQty',
       'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty',
       'stringLiteralsQty', 'numbersQty', 'assignmentsQty',
       'mathOperationsQty', 'variablesQty', 'maxNestedBlocks',
       'anonymousClassesQty', 'subClassesQty', 'lambdasQty', 'uniqueWordsQty',
       'modifiers', 'latest_commit', 'commit_compared_with', 'num_dependency',
       'num_line_affected', 'actual_num_of_classes_touched',
       'latest_commit_date', 'commit_compared_with_date', 'author',
       'author_email'],
      dtype='object')

In [788]:
columns_to_agg = [
    'cbo', 'wmc', 'dit', 'rfc', 'lcom',
       'totalMethods', 'staticMethods', 'publicMethods', 'privateMethods',
       'protectedMethods', 'defaultMethods', 'abstractMethods', 'finalMethods',
       'synchronizedMethods', 'totalFields', 'staticFields', 'publicFields',
       'privateFields', 'protectedFields', 'defaultFields', 'finalFields',
       'synchronizedFields', 'nosi', 'loc', 'returnQty', 'loopQty',
       'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty',
       'stringLiteralsQty', 'numbersQty', 'assignmentsQty',
       'mathOperationsQty', 'variablesQty', 'maxNestedBlocks',
       'anonymousClassesQty', 'subClassesQty', 'lambdasQty', 'uniqueWordsQty',
       'modifiers', 'num_dependency',
       'num_line_affected', 'actual_num_of_classes_touched'
]


project_refactoring_data_agg_column_names = ['commit_compared_with']
for element in columns_to_agg:
    project_refactoring_data_agg_column_names.append(element + '_mean')
    project_refactoring_data_agg_column_names.append(element + '_min')
    project_refactoring_data_agg_column_names.append(element + '_max')
    project_refactoring_data_agg_column_names.append(element + '_std')


In [789]:
project_refactoring_data_agg = project_refactoring_data.groupby(['commit_compared_with'], as_index=False).agg({
       'cbo': ['mean', 'min', 'max','std'], 
        'wmc': ['mean', 'min', 'max','std'], 
        'dit': ['mean', 'min', 'max','std'], 
        'rfc': ['mean', 'min', 'max','std'], 
        'lcom': ['mean', 'min', 'max','std'],
        'totalMethods': ['mean', 'min', 'max','std'], 
        'staticMethods': ['mean', 'min', 'max','std'],
        'publicMethods': ['mean', 'min', 'max','std'], 
        'privateMethods': ['mean', 'min', 'max','std'],
        'protectedMethods': ['mean', 'min', 'max','std'], 
        'defaultMethods': ['mean', 'min', 'max','std'], 
        'abstractMethods': ['mean', 'min', 'max','std'], 
        'finalMethods': ['mean', 'min', 'max','std'],
        'synchronizedMethods': ['mean', 'min', 'max','std'],
        'totalFields': ['mean', 'min', 'max','std'], 
        'staticFields': ['mean', 'min', 'max','std'], 
        'publicFields': ['mean', 'min', 'max','std'],
        'privateFields': ['mean', 'min', 'max','std'],
        'protectedFields': ['mean', 'min', 'max','std'],
        'defaultFields': ['mean', 'min', 'max','std'], 
        'finalFields': ['mean', 'min', 'max','std'],
        'synchronizedFields': ['mean', 'min', 'max','std'],
        'nosi': ['mean', 'min', 'max','std'], 
        'loc': ['mean', 'min', 'max','std'], 
        'returnQty': ['mean', 'min', 'max','std'],
        'loopQty': ['mean', 'min', 'max','std'],
        'comparisonsQty': ['mean', 'min', 'max','std'],
        'tryCatchQty': ['mean', 'min', 'max','std'],
        'parenthesizedExpsQty': ['mean', 'min', 'max','std'],
        'stringLiteralsQty': ['mean', 'min', 'max','std'],
        'numbersQty': ['mean', 'min', 'max','std'], 
        'assignmentsQty': ['mean', 'min', 'max','std'],
        'mathOperationsQty': ['mean', 'min', 'max','std'], 
        'variablesQty': ['mean', 'min', 'max','std'], 
        'maxNestedBlocks': ['mean', 'min', 'max','std'],
        'anonymousClassesQty': ['mean', 'min', 'max','std'],
        'subClassesQty': ['mean', 'min', 'max','std'],
        'lambdasQty': ['mean', 'min', 'max','std'],
        'uniqueWordsQty': ['mean', 'min', 'max','std'],
        'modifiers': ['mean', 'min', 'max','std'],
        'num_dependency': ['mean', 'min', 'max','std'],
        'num_line_affected': ['mean', 'min', 'max','std'],
        'actual_num_of_classes_touched': ['mean', 'min', 'max','std']
})


In [790]:
project_refactoring_data_agg.columns = project_refactoring_data_agg_column_names

In [791]:
project_refactoring_data_agg.head()

,commit_compared_with,cbo_mean,cbo_min,cbo_max,cbo_std,wmc_mean,wmc_min,wmc_max,wmc_std,dit_mean,...,num_dependency_max,num_dependency_std,num_line_affected_mean,num_line_affected_min,num_line_affected_max,num_line_affected_std,actual_num_of_classes_touched_mean,actual_num_of_classes_touched_min,actual_num_of_classes_touched_max,actual_num_of_classes_touched_std
0,083315a473d8b2fc972a70449272b296683ac38c,30.363636,20,66,13.574039,51.363636,12,174,47.535824,1.272727,...,777.0,306.293353,745.545455,129,2602,723.503471,26.181818,2,48,19.605194
1,334eb0072512155e57a809b2fa18861c0d68ec9f,8.500000,2,17,5.680376,13.250000,7,19,4.753946,1.250000,...,143.0,58.052849,117.500000,31,198,61.085732,4.500000,1,8,2.366432
2,3e331c108905a97fa9718b40844ddc1356fc86b5,11.250000,7,22,6.692213,24.000000,4,54,22.309511,1.500000,...,321.0,137.772068,217.500000,67,566,217.340418,4.500000,1,8,2.449490
3,4036fa55b72faaad3a14f806094184ade157917d,8.500000,6,11,3.535534,1.500000,1,2,0.707107,3.000000,...,9.0,3.535534,24.000000,15,33,12.727922,3.500000,1,6,3.535534
4,4677beea96f0afb1e061f119e52e0203d4cd3738,19.000000,19,19,NaN,17.000000,17,17,NaN,1.000000,...,14.0,NaN,295.000000,295,295,NaN,2.000000,2,2,NaN


In [792]:
final_df_agg = pd.merge(final_df, project_refactoring_data_agg, how='inner', left_on='sha', right_on='commit_compared_with')

In [793]:
final_df_agg.head()

,kmean_label,time_taken_mean,time_taken_min,time_taken_max,time_taken_q10,time_taken_q20,time_taken_q25,time_taken_q30,time_taken_q40,time_taken_q50,...,num_dependency_max,num_dependency_std,num_line_affected_mean,num_line_affected_min,num_line_affected_max,num_line_affected_std,actual_num_of_classes_touched_mean,actual_num_of_classes_touched_min,actual_num_of_classes_touched_max,actual_num_of_classes_touched_std
0,6,24.722772,1.0,148.0,2.0,3.0,4.0,6.0,10.4,14.0,...,777.0,306.293353,745.545455,129,2602,723.503471,26.181818,2,48,19.605194
1,6,24.722772,1.0,148.0,2.0,3.0,4.0,6.0,10.4,14.0,...,36.0,7.365460,366.750000,128,907,362.783108,5.000000,2,8,2.581989
2,7,24.990354,1.0,167.0,1.0,2.0,3.0,5.0,8.0,12.0,...,321.0,157.684284,323.250000,71,571,281.001038,20.750000,2,40,15.649814
3,7,24.990354,1.0,167.0,1.0,2.0,3.0,5.0,8.0,12.0,...,777.0,418.578945,800.333333,344,1506,619.827664,62.333333,39,82,21.733231
4,7,24.990354,1.0,167.0,1.0,2.0,3.0,5.0,8.0,12.0,...,11.0,NaN,3932.000000,3932,3932,NaN,2.000000,2,2,NaN


In [794]:
len(final_df)

9

In [795]:
len(final_df_agg)

8

In [796]:
final_df_agg.to_csv(f'Effort_Estimation_Results_v6/{project_name}.csv', index=False)

In [797]:
for column in final_df_agg.columns:
    print(column)

kmean_label
time_taken_mean
time_taken_min
time_taken_max
time_taken_q10
time_taken_q20
time_taken_q25
time_taken_q30
time_taken_q40
time_taken_q50
time_taken_q60
time_taken_q70
time_taken_q75
time_taken_q80
time_taken_q90
sha
name
email
date
login
message
parent_sha
parent_date
time_taken
contains_refactoring
project_name
commit_compared_with
cbo_mean
cbo_min
cbo_max
cbo_std
wmc_mean
wmc_min
wmc_max
wmc_std
dit_mean
dit_min
dit_max
dit_std
rfc_mean
rfc_min
rfc_max
rfc_std
lcom_mean
lcom_min
lcom_max
lcom_std
totalMethods_mean
totalMethods_min
totalMethods_max
totalMethods_std
staticMethods_mean
staticMethods_min
staticMethods_max
staticMethods_std
publicMethods_mean
publicMethods_min
publicMethods_max
publicMethods_std
privateMethods_mean
privateMethods_min
privateMethods_max
privateMethods_std
protectedMethods_mean
protectedMethods_min
protectedMethods_max
protectedMethods_std
defaultMethods_mean
defaultMethods_min
defaultMethods_max
defaultMethods_std
abstractMethods_mean
abstractMe